In [2]:
# import all necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
!pip install lxml
print('Libraries imported.')

Libraries imported.


In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipage = requests.get(url)
wikipage.text[:100]

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<titl'

In [4]:
toronto = pd.DataFrame()
soup = BeautifulSoup(wikipage.text, 'lxml')
wikitable = soup.find_all('table')[0]

In [ ]:
row_marker = 0

for row in wikitable.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        toronto.loc[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

#rename column names
toronto.rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'}, inplace=True)
#drop all rows with unassigned borough
toronto = toronto[toronto.Borough!='Not assigned']
toronto.reset_index(drop=True,inplace=True)

#create a new dataframe toronto_neighbor to combine all neighborhoods with the same postal code
toronto_data= pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])

for ii in range(toronto.shape[0]):
    borough = toronto.loc[ii,'Borough']
    postalcode = toronto.loc[ii,'PostalCode']
    neighborhood = toronto.loc[ii,'Neighborhood'][:-1]
    if neighborhood == 'Not assigned':
        neighborhood = borough
    
    if postalcode in toronto_neighbor.PostalCode.values:
        old_neighborhood = toronto_neighbor[toronto_neighbor.PostalCode == postalcode].Neighborhood
        new_value = (old_neighborhood + ',' + neighborhood)
        toronto_neighbor.loc[toronto_neighbor.PostalCode == postalcode,'Neighborhood'] = new_value 
    
    else:
        toronto_neighbor = toronto_neighbor.append({'PostalCode': postalcode,
                                                'Borough': borough,
                                                'Neighborhood': neighborhood,
                                               }, ignore_index=True)
toronto_neighbor[['PostalCode','waste']] = toronto_neighbor.PostalCode.str.split("\n",expand=True)
toronto_neighbor[['Borough','waste1']] = toronto_neighbor.Borough.str.split("\n",expand=True)
toronto_neighbor[['Neighborhood','waste']] = toronto_neighbor.Neighborhood.str.split("\n",expand=True)
toronto_neighbor.drop(['waste1'],axis=1,inplace=True)

In [25]:
toronto_neighbor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
toronto_neighbor.shape

(180, 3)